In [45]:
import os
import json
from PIL import Image
from tqdm import tqdm
import time

In [46]:
def crop_and_resize_and_save_face(json_path, file_dir_path, save_dir, label, cropped_shape = (256,256)):
    #target데이터
    with open(json_path, 'r') as json_file:
        data = json.load(json_file)
    
    data_dict = {item["filename"]: item for item in data}

    #저장 될 폴더가 존재하지 않으면 생성
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    processed_count = 0
    error_count = 0

    for filename in tqdm(os.listdir(file_dir_path),desc=f"processing label {label}"):
        file_path = os.path.join(file_dir_path, filename)

        if filename in data_dict:
            item = data_dict[filename]

            try:
                with Image.open(file_path) as img:
                    face_coords = item["annot_A"]["boxes"]
                    left = face_coords["minX"]
                    right = face_coords["maxX"]
                    top = face_coords["minY"]
                    bottom = face_coords["maxY"]
                    #유효성 검사
                    if left >= 0 and top >= 0 and right <= img.width and bottom <= img.height and right > left and bottom > top:
                        cropped_img = img.crop((left, top, right, bottom))

                        if cropped_img.size != (0, 0):
                            resized_img = cropped_img.resize(cropped_shape)
                            temp_filename = f"{label}_{filename}"
                            resized_img.save(os.path.join(save_dir,temp_filename))
                            processed_count += 1
                            # print(f"{processed_count} - Successfully cropped and saved {temp_filename}")
                        else:
                            # print(f"Empty cropped image for {filename}")
                            error_count += 1
                    else:
                        # print(f"Invalid crop coordinates for {filename}")
                        error_count += 1
            except Exception as e:
                print(f"Error: {e}")
                error_count += 1
    
    return processed_count , error_count

In [47]:
classes = ["분노","슬픔","불안","상처","당황","기쁨","중립"]
processed_list = []
error_list = []
start = time.time()
for class_index,class_ in enumerate(classes):
    json_path = f"Training\[라벨]EMOIMG_{class_}_TRAIN\img_emotion_training_data({class_}).json"
    file_dir_path = f"Training\[원천]EMOIMG_{class_}_TRAIN_01"
    save_dir = "Training/Cropped_Images"
    (processed_count, error_count) = crop_and_resize_and_save_face(json_path, file_dir_path, save_dir,label = class_index,cropped_shape=(256,256))
    print(processed_count, error_count)
    processed_list.append(processed_count)
    error_list.append(error_count)
end = time.time()

print(f"걸린 시간: {round(end-start,2)}s")
for class_,processed_count,error_count in zip(classes,processed_list,error_list):
    print(class_,f"total_count = {processed_count}, error_count = {error_count}")

processing label 0: 100%|██████████| 16234/16234 [12:16<00:00, 22.04it/s] 


16117 117


processing label 1: 100%|██████████| 15874/15874 [12:10<00:00, 21.74it/s] 


15787 87


processing label 2: 100%|██████████| 16133/16133 [12:17<00:00, 21.87it/s] 


...

Error: broken data stream when reading image file


processing label 6:  57%|█████▋    | 9182/16197 [06:56<06:07, 19.11it/s]

Error: broken data stream when reading image file
Error: broken data stream when reading image file


processing label 6: 100%|██████████| 16197/16197 [12:05<00:00, 22.34it/s]


16074 123
걸린 시간: 5151.46s
분노 total_count = 16117, error_count = 117
슬픔 total_count = 15787, error_count = 87
불안 total_count = 16046, error_count = 87
상처 total_count = 15930, error_count = 93
당황 total_count = 16081, error_count = 89
기쁨 total_count = 15984, error_count = 88
중립 total_count = 16074, error_count = 123
